In [1]:
!pip install SpeechRecognition gtts playsound pyjokes wikipedia pyaudio pygame

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/32.9 MB ? eta -:--:--
   ---------------------------------------- 0.2/32.9 MB 4.8 MB/s eta 0:00:07
    --------------------------------------- 0.6/32.9 MB 6.8 MB/s eta 0:00:05
   - -------------------------------------- 1.2/32.9 MB 8.2 MB/s eta 0:00:04
   - -------------------------------------- 1.5/32.9 MB 8.0 MB/s eta 0:00:04
   -- ------------------------------------- 2.0/32.9 MB 8.5 MB/s eta 0:00:04
   -- ------------------------------------- 2.4/32.9 MB 8.5 MB/s eta 0:00:04
   --- ------------------------------------ 2.9/32.9 MB 8.8 MB/s eta 0:00:04
   ---- ----------------------------------- 3.3/32.9 MB 8.9 MB/s eta 0:00:04
   ---- ----------------------------------- 3.8/32.9 MB 9.0 MB/s eta 0:00:04
   ----- ------------------------

In [26]:
import speech_recognition as sr
from gtts import gTTS
import os
import tempfile
from datetime import datetime
import pyjokes
import wikipedia
import webbrowser
from pygame import mixer
import platform
import time

In [27]:
# Configurações iniciais
wikipedia.set_lang("pt")  # Define o idioma da Wikipedia para português
mixer.init()  # Inicializa o mixer do pygame para reprodução de sons (chamado uma única vez)

In [30]:
def falar(texto):
    """Faz a assistente falar um texto usando gTTS."""
    with tempfile.NamedTemporaryFile(suffix=".mp3", delete=False) as temp_audio:
        caminho_audio = temp_audio.name

    tts = gTTS(texto, lang="pt-br")
    tts.save(caminho_audio)
    
    mixer.music.load(caminho_audio)
    mixer.music.play()
    while mixer.music.get_busy():  # Espera o áudio terminar
        continue
    mixer.music.unload()  # Libera o arquivo de áudio
    
    # Exclui o arquivo após a reprodução
    if os.path.exists(caminho_audio):
        os.remove(caminho_audio)

def ouvir_comando():
    """Ouve o comando de voz do usuário."""
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Ouvindo...")
        recognizer.adjust_for_ambient_noise(source)  # Ajusta para o ruído ambiente
        audio = recognizer.listen(source)

        try:
            comando = recognizer.recognize_google(audio, language="pt-BR")
            print(f"Você disse: {comando}")
            return comando.lower()
        except sr.UnknownValueError:
            falar("Desculpe, não entendi o comando.")
            return None
        except sr.RequestError:
            falar("Desculpe, o serviço de reconhecimento de voz está fora do ar.")
            return None

def limpar_lixeira():
    """Limpa a lixeira (funciona apenas no Windows)."""
    sistema = platform.system()
    if sistema == "Windows":
        import winshell
        try:
            winshell.recycle_bin().empty(confirm=False, show_progress=False, sound=True)
            falar("Lixeira limpa.")
        except Exception as e:
            falar("Não foi possível limpar a lixeira.")
    else:
        falar("Desculpe, essa funcionalidade só está disponível no Windows.")

def executar_comando(comando):
    """Executa o comando recebido."""
    if 'abrir youtube' in comando:
        falar("Abrindo o YouTube.")
        webbrowser.open("https://www.youtube.com")
    elif 'abrir google' in comando:
        falar("Abrindo o Google.")
        webbrowser.open("https://www.google.com")
    elif 'tocar música' in comando:
        falar("Tocando música.")
        mixer.music.load("caminho_para_sua_musica.mp3")  # Coloque o caminho da sua música
        mixer.music.play()
    elif 'hora' in comando:
        hora = datetime.now().strftime('%H:%M')
        falar(f"Agora são {hora}")
    elif 'piada' in comando:
        piada = pyjokes.get_joke(language='pt')
        falar(piada)
    elif 'pesquisar' in comando:
        termo = comando.replace('pesquisar', '').strip()
        falar(f"Pesquisando {termo} na Wikipedia.")
        try:
            resultado = wikipedia.summary(termo, sentences=2)
            falar(resultado)
        except wikipedia.exceptions.DisambiguationError as e:
            falar("Encontrei vários resultados. Por favor, seja mais específico.")
        except wikipedia.exceptions.PageError:
            falar("Desculpe, não encontrei resultados para essa pesquisa.")
    elif 'limpar lixeira' in comando:
        limpar_lixeira()
    elif 'sair' in comando or 'parar' in comando:
        falar("Encerrando assistente")
        exit()
    else:
        falar("Comando não reconhecido.")

if __name__ == "__main__":
    falar("Olá! Sou sua assistente virtual. Como posso ajudar?")
    while True:
        comando = ouvir_comando()
        if comando:
            executar_comando(comando)

Ouvindo...
Você disse: encerrando
Ouvindo...
Ouvindo...
Você disse: encerrando assiste
Ouvindo...
Você disse: encerrando assistente
Ouvindo...
Ouvindo...
Você disse: Abrir o YouTube
Ouvindo...


KeyboardInterrupt: 